# Lobito Corridor Spatial Analysis
----

Benny Istanto, bistanto@worldbank.org, GOST/DEC Data Group

Objective. Produce fast, reproducible **access & opportunity** analytics using only aligned rasters: 
1. Isochrone coverage KPIs (population, cropland, electrification), 
2. A transparent **priority surface** for smallholder aggregation/feeder upgrades, 
3. A **flood-aware road risk** screening, and 
4. **Site cards** for the project locations.
              
**Expected outputs.** AOI-prefixed GeoTIFFs in `outputs/rasters/`, CSV tables in `outputs/tables/`, and quick PNG maps in `outputs/figs/`.

## Run once after (re)starting the kernel


<div class="alert alert-block alert-success" align="left">
  <b>Why this cell?</b><br/>
  It initializes the workspace for the current AOI—sets <code>PROJECT_ROOT</code>/<code>AOI</code>, adds <code>src/</code> to <code>sys.path</code>, clears the import cache, and loads <code>config.py</code> so <code>PATHS</code>/<code>PARAMS</code> are available to all steps.
  <br/><br/>
  <b>When to run it</b>
  <ul>
    <li>After a kernel restart (once).</li>
    <li>After changing <code>AOI_VALUE</code> or editing <code>config.py</code> (paths/parameters/templates).</li>
    <li>After moving the project folder.</li>
  </ul>
  <b>Editing step code?</b> Re-import the step and <code>importlib.reload(module)</code> before calling <code>main()</code>.
</div>


In [1]:
# Bootstrap imports for package-style repo
import os, sys, importlib
from pathlib import Path

# >>>> EDIT THESE TWO ONLY <<<<
PROJECT_ROOT = Path("/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis")
AOI_VALUE = "huambo"  # e.g., "benguela", "huambo", "bei", "moxico", "moxicoleste"

# Environment for the codebase
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
os.environ["AOI"] = AOI_VALUE

src_path = PROJECT_ROOT / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Make sure Python sees any file edits we made
importlib.invalidate_caches()

from config import PATHS, PARAMS, AOI, get_logger
print("AOI:", AOI)
PATHS.OUT, PATHS.OUT_R, PATHS.OUT_T, PATHS.OUT_F


AOI: huambo


(PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/rasters'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/figs'))

## Step 00 — Data alignment & vector→raster

Align everything to a single 1-km template so every pixel stacks perfectly. This step converts key vectors (cropland, electrification, settlements) to rasters, harmonizes units/CRS, and prepares a 1-km flood screening layer alongside the native 30 m flood depth for engineering checks.

**Objective.** Produce AOI-aligned base rasters and vector-to-raster layers for consistent downstream math.

**Inputs.**
* `PARAMS.TARGET_GRID` (template), raw rasters (POP/NTL/VEG/drought), flood depth (30 m), and vectors (cropland, electrification, settlements).
  
**Outputs.**
* `{AOI}_pop_1km.tif`, `{AOI}_ntl_1km.tif`, `{AOI}_veg_1km.tif`, `{AOI}_drought_1km.tif`,
* `{AOI}_cropland_1km.tif`, `{AOI}_electric_1km.tif`, `{AOI}_settlement_1km.tif`,
* `{AOI}_flood_maxdepth_30m.tif` + `{AOI}_flood_maxdepth_1km.tif`,
* `{AOI}_rwi_meta_1km.tif` (if available in your data bundle).
  
**Assumptions & design.**
* Everything is reprojected/matched to the template; vector rasterization uses `all_touched=True`.
* Nodata is respected; logging warns on missing optional layers.

**Knobs.** 
* In `config.py`: source paths, `PARAMS.TARGET_GRID`.


In [2]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_00_align_and_rasterize"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:39:15 | INFO | Loaded target grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083 (units of CRS)
06:39:15 | INFO | Aligning rasters to target grid...
06:39:16 | INFO | Wrote huambo_pop_1km.tif
06:39:16 | INFO | Wrote huambo_ntl_1km.tif
06:39:16 | INFO | Wrote huambo_veg_1km.tif
06:39:16 | INFO | Wrote huambo_drought_1km.tif
06:39:16 | INFO | Aligning Meta Relative Wealth Index (RWI) to 1-km grid...
06:39:16 | INFO | Wrote huambo_rwi_meta_1km.tif
06:39:16 | INFO | Rasterizing cropland presence (1=any in cell)...
06:40:50 | INFO | Wrote huambo_cropland_presence_1km.tif
06:40:50 | INFO | Rasterizing cropland fraction (0..1 per 1-km cell, supersample=10)...
06:42:11 | INFO | Wrote huambo_cropland_fraction_1km.tif | mean_fraction=0.252
06:42:16 | INFO | Rasterizing electricity masks using field 'FinalElecC' (1=grid, 99=unelectrified)...
06:42:23 | INFO | Wrote huambo_elec_grid_1km.tif, huambo_elec_unelectrified_1km.tif
06:42:28 | INFO | Rasterizing settlement type using field 'IsUrba

## Step 01 — Isochrones (accumulated travel time)

Turn road/friction inputs into a travel-time surface in minutes. This becomes the backbone for coverage masks (≤30/60/120 min) and access KPIs.

**Objective.** Compute the accumulated cost (minutes) raster.
                                                                                                               
**Inputs.**
* `PARAMS.TARGET_GRID`, friction/network inputs per `config.py`.
  
**Outputs.**
* `{AOI}_traveltime_min.tif` (accumulated cost surface).
  
**Assumptions.**
* Minutes as the canonical unit; obstacles/penalties embedded in friction.
  
**Knobs.**
* Friction parameters in `PARAMS` (if exposed), thresholds in `PARAMS.ISO_THRESH` (e.g., `(30, 60, 120)`).


In [3]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_01_isochrones"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:38 | INFO | Isochrones from ago_phy_huambo_traveltime_market.tif | CRS=EPSG:4326 | size=280x216 | cell=0.0x0.0
06:42:38 | INFO | Building ≤30 min isochrone...
06:42:38 | INFO | Wrote huambo_iso_le_30min_1km.tif | cells inside: 2230/39751 (5.6%)
06:42:38 | INFO | Building ≤60 min isochrone...
06:42:38 | INFO | Wrote huambo_iso_le_60min_1km.tif | cells inside: 7215/39751 (18.2%)
06:42:38 | INFO | Building ≤120 min isochrone...
06:42:38 | INFO | Wrote huambo_iso_le_120min_1km.tif | cells inside: 21964/39751 (55.3%)
06:42:38 | INFO | Building ≤240 min isochrone...
06:42:39 | INFO | Wrote huambo_iso_le_240min_1km.tif | cells inside: 36738/39751 (92.4%)
06:42:39 | INFO | Step 01 complete.


## Step 02 — KPIs for isochrone coverage (population, cropland, electrification)

Quantify how much population, cropland, and electrification lie within each time threshold. These coverage KPIs are the simplest way to communicate “who’s connected.”

**Objective.** Summarize POP/CROP/ELECTRIC inside `tt ≤ threshold`.

**Inputs.**
* `{AOI}_traveltime_min.tif`, 1-km POP/CROPLAND/ELECTRIC rasters.
  
**Outputs.**
* `tables/{AOI}_iso_kpis.csv` (per threshold).
  
**Assumptions.**
* Coverage mask: `tt_minutes <= threshold`.
  
**Knobs.**
* `PARAMS.ISO_THRESH` list.


In [4]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_02_kpis_population_cropland_electric"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:39 | INFO | Electrification grid value counts (non-NaN): {np.int64(1): 588}
06:42:39 | INFO | KPI base rasters loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:39 | INFO | Electrification mapping: FinalElecCode2020 -> elec_grid_1km == 1 (grid); 99/0/NaN = non-grid.
06:42:39 | INFO | Denominators | pop_total=2,997,197 | cropland_total_km2=10,156.91 | electrified_cells_total=588 | cell_area_km2≈1.000
06:42:39 | INFO | ≤ 30 min | cells=2,230 (area=2,230.00 km²) | pop=1,301,172 (43.4%) | crop=879.63 km² (8.7%) | elec_cells=414 (70.4%)
06:42:39 | INFO | ≤ 60 min | cells=7,215 (area=7,215.00 km²) | pop=1,635,860 (54.6%) | crop=2,212.28 km² (21.8%) | elec_cells=448 (76.2%)
06:42:39 | INFO | ≤120 min | cells=21,964 (area=21,964.00 km²) | pop=2,438,910 (81.4%) | crop=5,996.37 km² (59.0%) | elec_cells=563 (95.7%)
06:42:39 | INFO | ≤240 min | cells=36,738 (area=36,738.00 km²) | pop=2,920,311 (97.4%) | crop=9,485.93 km² (93.4%) | elec_cells=584 (99.3%)
06:42:39 | INFO | Saved

## Step 03 — Priority surface (baseline)

Combine normalized indicators: better access (lower minutes), higher population & vegetation, stronger nighttime lights, and lower drought frequency. Save the score raster and a top-decile mask to highlight quick-win areas.

**Objective.** (Baseline) Build a transparent, minimal priority surface from core overlays.

**Inputs.**
* 1-km standardized layers (POP/CROP/ELECTRIC etc.).
  
**Outputs.**
* `rasters/{AOI}_priority_score_0_1.tif`, `rasters/{AOI}_priority_top10_mask.tif`.
  
**Assumptions.**
* Simpler normalization and weights vs Step 07.
  
**Recommendation.**
* Keep Step 03 as **baseline** only; prefer Step 07 for production.


In [5]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_03_priority_surface"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:39 | INFO | Priority surface inputs loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:39 | INFO | Weights | ACC=0.35 POP=0.25 VEG=0.20 NTL=0.10 DRT=0.10
06:42:39 | INFO | Wrote huambo_priority_score_v1_0_1.tif
06:42:39 | INFO | Wrote huambo_priority_top10_mask_v1.tif | P90=0.763 | top10 cells: 446/4455 (10.0%)
06:42:39 | INFO | Step 03 complete.


## Step 04 — Flood bottlenecks from road raster

Flag where roads intersect flood depths beyond thresholds. Use the 30 m layer for engineering plausibility, and the 1 km “max depth” for corridor-scale screening.

**Objective.** Identify likely flood bottlenecks impacting road access.

**Inputs.**
* 30 m flood depth, road raster or vector.
  
**Outputs.**
* `rasters/{AOI}_flood_bottlenecks_1km.tif` (or CSV summary, depending on implementation).
  
**Assumptions.**
* Engineering check: depth thresholds configurable.
  
**Knobs.**
* Flood depth threshold(s) in `PARAMS` (if exposed).


In [6]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_04_flood_bottlenecks_from_road_raster"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:39 | WARNING | Reprojected to match target grid: flood1k, flood_frac, prio10
06:42:39 | INFO | Flood screening inputs loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:39 | INFO | Params | FLOOD_DEPTH_RISK=0.3 m | ROAD_CLASSES_KEEP=None | ROADS_ALL_TOUCHED=False
06:42:39 | INFO | Rasterizing roads: no class filter (use all OSM fclass values).
06:42:42 | INFO | Wrote huambo_roads_main_1km.tif | road_cells=12,523
06:42:42 | INFO | Wrote huambo_roads_flood_risk_cells_1km.tif | risk_cells=727
06:42:42 | INFO | Wrote huambo_roads_flood_risk_near_priority_1km.tif | risk_near_priority_cells=67
06:42:42 | INFO | Saved summary → /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/huambo_roads_flood_risk_summary.csv
06:42:42 | INFO | Step 04 complete.


## Step 05 — Site audit points

Build “site cards” by sampling rasters around project points. Quicklook stats and thumbnails help triage which locations deserve deeper design work.

**Objective.** Generate site-level diagnostic metrics and snapshots.
    
**Inputs.**
* Project points (CSV/GeoPackage as per `config.py`).
  
**Outputs.**
* `tables/{AOI}_site_audit_points.csv` (+ optional PNGs in `outputs/figs/`).
  
**Assumptions.**
* Points in EPSG:4326; rasters sampled with nearest or bilinear.
  
**Knobs.**
* Sampling radius / list of rasters to sample (if exposed).


In [7]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_05_site_audit_points"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:42 | INFO | Site audit inputs loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:42 | INFO | Loaded 39 site(s) from ago_poi_huambo_projectloc_dm_p.shp
06:42:42 | INFO | Neighborhood kernel: radius=5 cell(s) | cells_in_kernel=81
06:42:42 | INFO | Saved site audit → /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/huambo_site_audit_points.csv | rows=39 | skipped_outside_grid=0
06:42:42 | INFO | Step 05 complete.


## Step 06 — Municipality (Admin2) ingest & correlations

Ingest Admin2 (RAPP) themes, standardize units, rasterize selected variables, and test correlations versus rural poverty. Produces a tidy “profiles” table and 1-km Admin2 surfaces usable in later steps.

**Objective.** Normalize Admin2 indicators and rasterize key variables.
    
**Inputs.**
* `data/vectors/ago_*_rapp_2020_a.shp` (themes in `THEME_VARS`), `PARAMS.TARGET_GRID`.
  
**Outputs.**
* `tables/{AOI}_municipality_indicators.csv` (wide),
* `tables/{AOI}_corr_with_rural_poverty.csv`,
* `tables/{AOI}_municipality_profiles.csv`,
* `rasters/{AOI}_muni_{theme}_{var}_1km.tif`.
  
**Assumptions & design.**
* Percentages normalized to 0–1 (`RAPP_PCT_IS_0_100=True`); travel time hours → minutes; `all_touched=True`.
* Missing variables skipped with warnings.
  
**Knobs.**
* `THEME_VARS` mapping, list of variables to rasterize.


In [8]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_06_muni_ingest"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:42 | INFO | Target grid loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:42 | INFO | Step 06 will skip themes: ('climevents',)
06:42:42 | INFO | Theme=waterresources: read=0.09s, rename=0.00s, normalize=0.00s, rows=11, cols=10
06:42:43 | INFO | Theme=communications: read=0.10s, rename=0.00s, normalize=0.01s, rows=11, cols=11
06:42:43 | INFO | Theme=infra: read=0.10s, rename=0.00s, normalize=0.01s, rows=11, cols=15
06:42:43 | INFO | Theme=foodinsecurity: read=0.09s, rename=0.00s, normalize=0.01s, rows=11, cols=14
06:42:43 | INFO | Theme=outflow: read=0.09s, rename=0.00s, normalize=0.01s, rows=11, cols=12
06:42:43 | INFO | Theme=poverty: read=0.10s, rename=0.00s, normalize=0.00s, rows=11, cols=8
06:42:43 | INFO | Theme=productions: read=0.10s, rename=0.00s, normalize=0.01s, rows=11, cols=15
06:42:43 | INFO | Converted traveltime theme hours→minutes for traveltime (columns: avg_hours_to_market_financial)
06:42:43 | INFO | Theme=traveltime: read=0.09s, rename=0.00s, n

## Step 07 — Priority surface (tunable, authoritative)

Create the **production** priority surface with coherent normalization, optional equity/poverty overlays, focal smoothing, tiny-cluster removal, and a Top-X% mask.

**Objective.** Build the authoritative priority map and mask with flexible weights.

**Inputs.**
* 1-km base layers + optional Admin2 rasters (auto-discovered), including `rwi_meta_1km` if present.
  
**Outputs.**
* `rasters/{AOI}_priority_score_0_1.tif`, `rasters/{AOI}_priority_top10_mask.tif`.
  
**Assumptions & design.**
* Unified normalization, masks, focal smoothing, small-cluster removal, Top-X selection.
  
**Knobs.**
* Weights and Top-X% in `PARAMS`/module (e.g., `PARAMS.TOP_X_PCT`).
  
**Note.**
* Downstream steps (11, 12) expect **these** canonical filenames from `config.py`.


In [9]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_07_priority_tunable"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config, utils_geo
# importlib.reload(config)
# importlib.reload(utils_geo)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above./
# importlib.reload(m)

m.main()


06:42:49 | INFO | Target grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:51 | INFO | Reprojected raster to match grid: POP
06:42:51 | INFO | Reprojected raster to match grid: VEG
06:42:51 | INFO | Reprojected raster to match grid: NTL
06:42:51 | INFO | Reprojected raster to match grid: DRT
06:42:51 | INFO | Reprojected raster to match grid: CROP
06:42:51 | INFO | Reprojected raster to match grid: RURAL
06:42:51 | INFO | Reprojected overlay to match grid: huambo_rwi_meta_1km.tif
06:42:52 | INFO | Reprojected overlay to match grid: huambo_muni_poverty_poverty_rural_1km.tif
06:42:52 | INFO | Reprojected overlay to match grid: huambo_muni_foodinsecurity_food_insec_scale_1km.tif
06:42:52 | INFO | Reprojected overlay to match grid: huambo_muni_traveltime_avg_hours_to_market_financial_1km.tif
06:42:52 | INFO | Priority weight blend → ACC:0.29, POP:0.21, DRT:0.08, POV:0.12, FOOD:0.08, MTT:0.08, RWI:0.12
06:42:52 | INFO | Wrote huambo_priority_score_0_1.tif
06:42:52 | INFO | Wrot

## Step 08 — Project-level KPIs

Aggregate access/exposure/equity metrics over project buffers or polygons. Useful for comparing alternatives under the same data stack.

**Objective.** Compute KPI summaries for each project footprint/buffer.
    
**Inputs.**
* Project geometries + aligned rasters (POP/CROP/ELECTRIC/PRIORITY/etc.).
  
**Outputs.**
* `tables/{AOI}_project_kpis.csv`.
  
**Assumptions.**
* Buffers in meters; stats computed with consistent nodata handling.
  
**Knobs.**
* Buffer distance; KPI list (in step module).


In [10]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_08_project_kpis"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config, utils_geo
# importlib.reload(config)
# importlib.reload(utils_geo)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:52 | INFO | Template grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:52 | INFO | Reprojected pop to match grid
06:42:52 | INFO | Reprojected cropf to match grid
06:42:52 | INFO | Reprojected grid to match grid
06:42:52 | INFO | Reprojected rural to match grid
06:42:52 | INFO | Reprojected prio to match grid
06:42:52 | INFO | Reprojected risk to match grid
06:42:52 | INFO | Reprojected pov to match grid
06:42:52 | INFO | Denominators | pop_total=2,997,196 | cropland_total_km2=8527.64 | electrified_cells_total=588 | cell_area_km2=0.840
06:42:52 | INFO | Loaded 39 project(s) from ago_poi_huambo_projectloc_dm_p.shp
06:42:53 | INFO | Saved project KPIs → huambo_project_kpis.csv | rows=39
06:42:53 | INFO | Step 08 complete.


## Step 09 — Municipality (Admin2) targeting

Rank Admin2 units using access (≤60/120 min), poverty/equity signals, and priority shares. Produces an actionable shortlist for piloting.

**Objective.** Score and rank Admin2s for investment targeting.

**Inputs.**
* Admin2 rasters and tabular indicators from Step 06 + travel-time surface.
  
**Outputs.**
* `tables/{AOI}_muni_targeting.csv` (+ optional choropleth figs).
  
**Assumptions.**
* Uses `%≤60/≤120m` coverage, priority mask share, etc.
  
**Knobs.**
* Thresholds in `PARAMS.ISO_THRESH`; ranking weights in the module.


In [11]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_09_muni_targeting"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:53 | INFO | Template grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:53 | INFO | Using admin2 geometry from ago_pop_huambo_adm2_poverty_rapp_2020_a.shp
06:42:53 | INFO | Saved municipality targeting table → huambo_priority_muni_rank.csv | rows=11
06:42:53 | INFO | Step 09 complete.


## Step 10 — Priority scenarios

Stress-test the priority model with alternative weight sets (e.g., stronger equity vs. market access) and compare outcomes.

**Objective.** Generate and record scenario variants of the priority surface.
    
**Inputs.**
* Same base overlays as Step 07, different weight/toggle sets.
  
**Outputs.**
* `rasters/{AOI}_priority_score_scnX.tif`, masks, and a `tables/{AOI}_priority_scenarios.csv`.
  
**Assumptions.**
* Scenarios defined in code or `PARAMS`.
  
**Knobs.**
* Scenario configs (weights/toggles) in the module or `PARAMS`.


In [12]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_10_priority_scenarios"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:53 | INFO | Target grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:54 | INFO | Using on-disk baseline: huambo_priority_top10_mask.tif
06:42:54 | INFO | Scenario [baseline] — ACC+POP+DRT, rural-only, min crop 5%, 3x3 smooth, Top10%
06:42:54 | INFO | Scenario [no_drought] — ACC+POP only; rural-only; min crop 5%; Top10%
06:42:54 | INFO | Scenario [veg_signal] — ACC+POP+VEG; rural-only; min crop 10%; Top10%
06:42:54 | INFO | Scenario [top_800km2] — ACC+POP+DRT, rural-only; select fixed 800 km²; mild smooth
06:42:54 | INFO | Wrote scenario summary → huambo_priority_scenarios_summary.csv | scenarios=4 | masks_saved=4
06:42:54 | INFO | Wrote scenarios sidecar → huambo_priority_scenarios.meta.json
06:42:54 | INFO | Step 10 complete.


## Step 11 — Priority clusters

Convert the Top-X% priority mask into connected clusters and compute cluster KPIs (population, cropland, RWI, and share within ≤30/60/120 min). Small, noisy blobs are removed using cell and km² thresholds.

**Objective.** Extract cluster polygons/labels and summarize them for shortlisting.
    
**Inputs.**
* `PRIORITY_TOP10_TIF` from `config.py`, `traveltime_min` + base rasters, optional `rwi_meta_1km`.
  
**Outputs.**
* `rasters/{AOI}_priority_clusters_1km.tif` (via `PRIORITY_CLUSTERS_TIF`),
* `tables/{AOI}_priority_clusters.csv`.
  
**Assumptions & design.**
* Prune tiny clusters by min cells and **km²** (now configurable via `PARAMS.MIN_CLUSTER_KM2`).
* Added KPIs: `share_le_30m/60m/120m` (uses `PARAMS.ISO_THRESH`).
  
**Knobs.**
* `PARAMS.MIN_CLUSTER_CELLS`, `PARAMS.MIN_CLUSTER_KM2`, `PARAMS.ISO_THRESH`.


In [13]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_11_priority_clusters"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:54 | INFO | Template grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:55 | INFO | Connected components found: 2
06:42:55 | INFO | Pruned clusters: kept=2 | removed=0 by cells<30 or km2<2.0
06:42:55 | INFO | Saved clusters → huambo_priority_clusters.csv | clusters=2
06:42:55 | INFO | Step 11 complete.


## Step 12 — Travel-time catchments

Build catchment masks for each threshold and compute POP/CROP/RWI KPIs plus mean travel minutes—a quick validation of how “tight” the catchments are.

**Objective.** Derive per-threshold catchment KPIs and diagnostics.
    
**Inputs.**
* `{AOI}_traveltime_min.tif`, 1-km POP/CROPLAND, optional `rwi_meta_1km`.
  
**Outputs.**
* `tables/{AOI}_catchments_kpis.csv` (via `CATCHMENTS_KPI_CSV`),
* per-threshold masks in `rasters/` (if the step writes them).
  
**Assumptions & design.**
* Coverage = `tt <= thr`; includes `mean_travel_min` diagnostic.
  
**Knobs.**
* `PARAMS.ISO_THRESH`.


In [14]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_12_traveltime_catchments"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:42:55 | INFO | Template grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:42:55 | INFO | Building friction raster (minutes per km) from OSM roads + off-road baseline...
06:42:57 | INFO | Wrote huambo_friction_min_per_km.tif
06:42:57 | INFO | Computed isochrones for site 1 at 15.52640,-12.85710
06:42:57 | INFO | Computed isochrones for site 2 at 15.59792,-12.80258
06:42:58 | INFO | Computed isochrones for site 3 at 16.22180,-12.61580
06:42:58 | INFO | Computed isochrones for site 4 at 16.23470,-12.42550
06:42:58 | INFO | Computed isochrones for site 5 at 15.65676,-12.72231
06:42:58 | INFO | Computed isochrones for site 6 at 15.59924,-12.72806
06:42:59 | INFO | Computed isochrones for site 7 at 15.79899,-13.03986
06:42:59 | INFO | Computed isochrones for site 8 at 15.82374,-12.89107
06:42:59 | INFO | Computed isochrones for site 9 at 15.83206,-12.87903
06:42:59 | INFO | Computed isochrones for site 10 at 15.71652,-12.55124
06:43:00 | INFO | Computed isochrones for site 11 at

## Step 13 — Synergies overlay

Intersect priority with constraints/opportunities (e.g., flood risk, grid proximity, conservation). Useful to surface “no-regret” or “high-risk” zones.

**Objective.** Produce an overlay table (and optional rasters) capturing key synergies/tradeoffs.

**Inputs.**
* Priority rasters, flood/constraint rasters, grid/market proximity if available.
  
**Outputs.**
* `tables/{AOI}_synergies_overlay.csv`, optionally composited rasters.
  
**Assumptions.**
* Binary or categorical overlay logic documented in the step.
  
**Knobs.**
* Overlay weights/toggles in the module.


In [20]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_13_synergies_overlay"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
importlib.reload(m)

m.main()


21:47:35 | INFO | Loaded 39 site(s).
21:47:35 | INFO | gov layer not found: ago_poi_huambo_projects_gov_p.shp
21:47:35 | INFO | Loaded 39 wb feature(s) from ago_poi_huambo_projects_wb_p.shp
21:47:35 | INFO | oth layer not found: ago_poi_huambo_projects_others_p.shp
21:47:35 | INFO | Saved site synergies → huambo_site_synergies.csv | rows=39
21:47:35 | INFO | Saved cluster synergies → huambo_cluster_synergies.csv | rows=2
21:47:35 | INFO | Step 13 complete.


## Step 14 — Origin-Destination (Admin2 gravity)

Sketch flows between Admin2 zones using a gravity model for corridor storytelling. Supports equity tilting (RWI), selectable impedance (exponential or power), distance cutoffs, and (optionally) doubly-constrained balancing.

**Objective.** Generate OD flows, zone attributes, and sampled agents for quick viz.
                                                                      
**Inputs.**
* `{AOI}_pop_1km.tif`, optional `rwi_meta_1km.tif`, Admin2 shapefile (`muni_path_for`).
  
**Outputs.**
* `tables/{AOI}_od_gravity.csv` (long: oi,dj,flow,dist_km),
* `tables/{AOI}_od_zone_attrs.csv` (zone masses + centroids),
* `tables/{AOI}_od_agents.csv` (sampled OD pairs for quick viz).
  
**Assumptions & design.**
* Geodesic centroid–centroid distances; **optional** RWI mass tilt;
* Impedance `f(D)` selectable (`exp` or `pow`); optional **doubly-constrained** via IPF; distance cutoff.
  
**Knobs (in `PARAMS`).**
* `OD_F`: `"exp"` or `"pow"`; `OD_LAMBDA` (exp), `OD_BETA` (pow),
* `OD_MAX_DIST_KM`, `OD_TRIPS_TOTAL`, `OD_USE_DOUBLY_CONSTRAINED`,
* `USE_RWI_IN_MASS`, `RWI_WEIGHT`, `OD_N_AGENTS`.


In [16]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_14_lite_od"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


06:43:09 | INFO | RWI found: using equity tilt.
06:43:09 | INFO | OD zones aligned by ADM2CD_c order | N=11
06:43:09 | INFO | Flows: total=1,000,000 | mean_dist_km=71.6 | within_cutoff%=100.0
06:43:09 | INFO | Wrote huambo_od_gravity.csv, huambo_od_zone_attrs.csv
06:43:09 | INFO | Wrote huambo_od_agents.csv (N=121)
06:43:09 | INFO | Step 14 complete.


## End of Calculation